# Cherry Pit Detection: Optimization Experiments

This notebook runs three optimization experiments to improve upon the current best model (94.05% accuracy):

1. **Differential Learning Rates** - Fine-tune deeper layers with layer-specific LRs
2. **Threshold Optimization** - Find optimal decision boundaries that minimize missed pits
3. **ResNet18 Backbone** - Test smaller/faster model with similar accuracy

**Current Best Model:**
- Architecture: ResNet50
- Accuracy: 94.05%
- Latency: 16.7ms (CPU)
- Training: Augmentation + Unnormalized (0-255)

**Goals:**
- Minimize missed pits (false negatives) - food safety priority
- Maintain inference latency < 30ms
- Improve accuracy beyond 94.05%

## Step 1: Environment Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Install dependencies
!pip install -q pyyaml scikit-learn matplotlib tqdm

# Clone training repository
!git clone https://github.com/usefulmove/cherries.git

# Clone dataset repository (shallow)
!git clone --depth 1 https://github.com/weshavener/cherry_classification.git

print("\n" + "="*60)
print("SETUP COMPLETE")
print("="*60)

## Step 2: Verify GPU and Data

In [ ]:
import torch
import os

# Check GPU - HARD STOP if not available
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if not torch.cuda.is_available():
    raise RuntimeError(
        "\n" + "!"*60 + "\n" +
        "GPU REQUIRED FOR TRAINING!\n" +
        "Go to: Runtime -> Change runtime type -> GPU\n" +
        "Then re-run this cell.\n" +
        "!"*60
    )

print(f"CUDA version: {torch.version.cuda}")
print(f"GPU: {torch.cuda.get_device_name(0)}")

# Verify data structure
data_root = "/content/cherry_classification/data"
print(f"\nData directory structure:")
!ls -lh {data_root}
!ls -lh {data_root}/train
!ls -lh {data_root}/val

# Count samples
train_clean = len(os.listdir(f"{data_root}/train/cherry_clean"))
train_pit = len(os.listdir(f"{data_root}/train/cherry_pit"))
val_clean = len(os.listdir(f"{data_root}/val/cherry_clean"))
val_pit = len(os.listdir(f"{data_root}/val/cherry_pit"))

print(f"\nTraining samples: {train_clean} clean, {train_pit} pit (total: {train_clean + train_pit})")
print(f"Validation samples: {val_clean} clean, {val_pit} pit (total: {val_clean + val_pit})")

In [ ]:
# === SMOKE TEST CONFIGURATION ===
# Set to True to verify the notebook works (runs 1 epoch, few batches)
# Set to False for actual training
DRY_RUN = False

if DRY_RUN:
    print("\n" + "!"*60)
    print("DRY RUN MODE ENABLED - Running Smoke Test")
    print("!"*60 + "\n")

## Experiment 1: Differential Learning Rates

**Hypothesis:** Training different layers with different learning rates will improve accuracy.

**Approach:**
- Early layers (layer1-3): Frozen or very low LR (1e-6) - preserve ImageNet features
- Late layer (layer4): Medium LR (1e-5) - adapt to cherry-specific features
- Classifier (fc): High LR (1e-3) - learn task-specific classification

**Expected:** +0.5-1.5% accuracy improvement

In [ ]:
import subprocess
from pathlib import Path

# Configuration
exp_name = "resnet50_differential_lr"
config_path = f"cherries/training/configs/{exp_name}.yaml"
data_root = "/content/cherry_classification/data"
drive_output = f"/content/drive/MyDrive/cherry_training/experiments/{exp_name}"

print("\n" + "="*60)
print(f"EXPERIMENT 1: Differential Learning Rates")
print("="*60)
print(f"Config: {config_path}")
print(f"Output: {drive_output}")
print("\n")

# NOTE: The standard train.py doesn't support parameter groups yet.
# We need to modify the training call to use custom parameter groups.
# For now, we'll use a custom training cell below instead of calling train.py

print("Setting up custom training with differential learning rates...")

In [ ]:
# Custom training with differential LR
import sys
sys.path.insert(0, 'cherries/training')

import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
import yaml

from src.data import get_dataloaders
from src.model import create_classifier, save_model_weights_only
from src.metrics import calculate_metrics, collect_predictions, print_metrics_summary

# Load base config
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Override paths
config['data']['root'] = data_root
config['checkpointing']['output_dir'] = drive_output

# Create output directory
Path(drive_output).mkdir(parents=True, exist_ok=True)

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Data loaders
train_loader, val_loader = get_dataloaders(
    data_root=config['data']['root'],
    batch_size=config['data']['batch_size'],
    num_workers=config['data']['num_workers'],
    input_size=config['data']['input_size'],
    augmentation=config['data']['augmentation'],
    normalize=config['data']['normalize'],
)

class_names = train_loader.dataset.classes
print(f"Classes: {class_names}")

# Create model
model = create_classifier(
    architecture='resnet50',
    num_classes=2,
    pretrained=True,
    freeze_backbone=False,
    device=device,
)

# Create optimizer with differential learning rates
print("\nSetting up differential learning rates:")
print("  - layer1-3: frozen")
print("  - layer4: 1e-5")
print("  - fc: 1e-3")

# Freeze early layers
for param in model.layer1.parameters():
    param.requires_grad = False
for param in model.layer2.parameters():
    param.requires_grad = False
for param in model.layer3.parameters():
    param.requires_grad = False

# Create parameter groups
optimizer = optim.Adam([
    {'params': model.layer4.parameters(), 'lr': 1e-5},
    {'params': model.fc.parameters(), 'lr': 1e-3},
], weight_decay=config['training']['weight_decay'])

# Loss function
criterion = nn.CrossEntropyLoss()

print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Training loop
num_epochs = config['training']['epochs']
if DRY_RUN:
    num_epochs = 1
    print(f"DRY RUN: Reduced epochs to {num_epochs}")

best_val_acc = 0.0

print(f"\n{'='*60}")
print(f"TRAINING: {num_epochs} epochs")
print(f"{'='*60}\n")

for epoch in range(num_epochs):
    epoch_start = datetime.now()
    
    # Training
    model.train()
    train_loss = 0.0
    batch_count = 0
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        if DRY_RUN and batch_idx >= 3:
            print("DRY RUN: Stopping training epoch after 3 batches")
            break
            
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        batch_count += 1
        
        if (batch_idx + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx+1}/{len(train_loader)}] Loss: {loss.item():.4f}")
    
    train_loss /= batch_count
    
    # Validation
    model.eval()
    
    # Custom collection for Dry Run to avoid full validation pass
    if DRY_RUN:
        y_true_list, y_pred_list, y_probs_list = [], [], []
        with torch.no_grad():
            for i, (images, labels) in enumerate(val_loader):
                if i >= 3: break
                images = images.to(device)
                outputs = model(images)
                probs = torch.softmax(outputs, dim=1)
                _, preds = torch.max(outputs, 1)
                y_true_list.extend(labels.cpu().numpy())
                y_pred_list.extend(preds.cpu().numpy())
                y_probs_list.extend(probs.cpu().numpy())
        import numpy as np
        y_true = np.array(y_true_list)
        y_pred = np.array(y_pred_list)
        y_probs = np.array(y_probs_list)
    else:
        y_true, y_pred, y_probs = collect_predictions(model, val_loader, device)
        
    val_metrics = calculate_metrics(y_true, y_pred, y_probs, class_names)
    
    print(f"\nEpoch {epoch+1}/{num_epochs}:")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Val Accuracy: {val_metrics['accuracy']:.4f}")
    print(f"  Val F1: {val_metrics['f1']:.4f}")
    
    # Save best model
    if val_metrics['accuracy'] > best_val_acc:
        best_val_acc = val_metrics['accuracy']
        best_path = Path(drive_output) / 'model_best.pt'
        save_model_weights_only(model, str(best_path))
        print(f"  *** New best model saved! Accuracy: {best_val_acc:.4f} ***")
    
    epoch_time = (datetime.now() - epoch_start).total_seconds()
    print(f"  Time: {epoch_time:.1f}s\n")

print(f"\n{'='*60}")
print(f"EXPERIMENT 1 COMPLETE")
print(f"Best Validation Accuracy: {best_val_acc:.4f}")
print(f"{'='*60}")

## Experiment 2: Threshold Optimization

**Hypothesis:** Current thresholds (pit≥0.75, clean≥0.5) are suboptimal for minimizing missed pits.

**Approach:**
- Analyze probability distributions on validation set
- Find threshold that achieves ≥95% pit recall (minimize false negatives)
- Balance against false positives (wasted good cherries)

**Expected:** Better pit detection with acceptable clean waste rate

In [ ]:
# Run threshold optimization on the best model from Experiment 1
print("\n" + "="*60)
print("EXPERIMENT 2: Threshold Optimization")
print("="*60)

model_path = f"{drive_output}/model_best.pt"
threshold_output = f"{drive_output}/threshold_analysis"

cmd = [
    "python", "cherries/training/scripts/optimize_thresholds.py",
    "--model-path", model_path,
    "--data-root", data_root,
    "--architecture", "resnet50",
    "--output-dir", threshold_output,
    "--min-recall", "0.95",  # Target: catch ≥95% of pits
    "--device", "cuda" if torch.cuda.is_available() else "cpu"
]

if DRY_RUN:
    cmd.append("--dry-run")
    print("DRY RUN: Added --dry-run flag")

print(f"\nRunning threshold optimization...")
print(f"Model: {model_path}")
print(f"Output: {threshold_output}")
print(f"Target: ≥95% pit recall\n")

result = subprocess.run(cmd, capture_output=False, text=True)

if result.returncode == 0:
    print(f"\n{'='*60}")
    print("EXPERIMENT 2 COMPLETE")
    print(f"{'='*60}")
    print(f"\nResults saved to: {threshold_output}")
    print("Check the generated plots and threshold_results.json")
else:
    print(f"\nThreshold optimization failed with return code {result.returncode}")

### Display Threshold Analysis Results

In [ ]:
from IPython.display import Image, display
import json

# Display plots
print("Threshold Analysis Plots:\n")

prob_dist_path = f"{threshold_output}/probability_distributions.png"
threshold_plot_path = f"{threshold_output}/threshold_analysis.png"

if Path(prob_dist_path).exists():
    print("Probability Distributions:")
    display(Image(filename=prob_dist_path))

if Path(threshold_plot_path).exists():
    print("\nThreshold Analysis:")
    display(Image(filename=threshold_plot_path))

# Load and display optimal thresholds
results_file = f"{threshold_output}/threshold_results.json"
if Path(results_file).exists():
    with open(results_file, 'r') as f:
        results = json.load(f)
    
    print("\nOptimal Thresholds Summary:")
    print("="*60)
    
    # Find threshold with best F1 that meets min_recall=0.95
    valid = [r for r in results if r['recall'] >= 0.95]
    if valid:
        best = max(valid, key=lambda x: x['f1'])
        print(f"\nRecommended Threshold: {best['threshold']:.3f}")
        print(f"  Accuracy:  {best['accuracy']:.4f}")
        print(f"  Precision: {best['precision']:.4f}")
        print(f"  Recall:    {best['recall']:.4f} (≥95% target met)")
        print(f"  F1 Score:  {best['f1']:.4f}")
        print(f"  Missed Pits (FN): {best['fn']}")
        print(f"  False Alarms (FP): {best['fp']}")
    else:
        print("No threshold meets 95% recall target - review plots")

## Experiment 3: ResNet18 Backbone

**Hypothesis:** ResNet18 (11.7M params) can achieve similar accuracy to ResNet50 (25.6M params) with faster inference.

**Approach:**
- Train ResNet18 with same augmentation/unnormalized setup
- Compare accuracy, model size, inference latency

**Expected:** ~94% accuracy, ~40% faster inference, smaller model size

In [ ]:
exp_name_r18 = "resnet18_augmented_unnormalized"
config_path_r18 = f"cherries/training/configs/{exp_name_r18}.yaml"
drive_output_r18 = f"/content/drive/MyDrive/cherry_training/experiments/{exp_name_r18}"

print("\n" + "="*60)
print("EXPERIMENT 3: ResNet18 Backbone")
print("="*60)
print(f"Config: {config_path_r18}")
print(f"Output: {drive_output_r18}")
print("\n")

cmd_r18 = [
    "python", "cherries/training/scripts/train.py",
    "--config", config_path_r18,
    "--output-dir", drive_output_r18,
    "--data-root", data_root
]

if DRY_RUN:
    cmd_r18.append("--dry-run")
    print("DRY RUN: Added --dry-run flag")

try:
    process = subprocess.Popen(
        cmd_r18,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    for line in process.stdout:
        print(line, end='')
    
    process.wait()
    
    if process.returncode == 0:
        print(f"\n{'='*60}")
        print("EXPERIMENT 3 COMPLETE")
        print(f"{'='*60}")
    else:
        print(f"\nTraining failed with return code {process.returncode}")
        
except Exception as e:
    print(f"\nError: {e}")

## Step 3: Benchmark Latency Comparison

Compare inference speed of ResNet50 (Exp 1) vs ResNet18 (Exp 3)

In [ ]:
import time
import torch
from src.model import create_classifier, load_model_weights_only

print("\n" + "="*60)
print("LATENCY BENCHMARK")
print("="*60)

device = 'cpu'  # Test on CPU for production comparison
num_iterations = 100

# Dummy input (matches production: 128x128, RGB, 0-255)
dummy_input = torch.randint(0, 256, (1, 3, 128, 128), dtype=torch.float32).to(device)

def benchmark_model(model_path, architecture, name):
    print(f"\nBenchmarking {name}...")
    
    model = create_classifier(
        architecture=architecture,
        num_classes=2,
        pretrained=False,
        device=device,
    )
    model = load_model_weights_only(model, model_path, device=device)
    model.eval()
    
    # Warmup
    for _ in range(10):
        with torch.no_grad():
            _ = model(dummy_input)
    
    # Benchmark
    start = time.time()
    with torch.no_grad():
        for _ in range(num_iterations):
            _ = model(dummy_input)
    end = time.time()
    
    avg_latency_ms = (end - start) / num_iterations * 1000
    
    # Model size
    param_count = sum(p.numel() for p in model.parameters())
    
    print(f"  Architecture: {architecture}")
    print(f"  Parameters: {param_count:,}")
    print(f"  Avg Latency: {avg_latency_ms:.2f}ms")
    
    return avg_latency_ms, param_count

# Benchmark ResNet50 (Exp 1)
r50_path = f"{drive_output}/model_best.pt"
if Path(r50_path).exists():
    r50_latency, r50_params = benchmark_model(r50_path, "resnet50", "ResNet50 (Differential LR)")
else:
    print(f"ResNet50 model not found: {r50_path}")
    r50_latency, r50_params = None, None

# Benchmark ResNet18 (Exp 3)
r18_path = f"{drive_output_r18}/model_best.pt"
if Path(r18_path).exists():
    r18_latency, r18_params = benchmark_model(r18_path, "resnet18", "ResNet18")
else:
    print(f"ResNet18 model not found: {r18_path}")
    r18_latency, r18_params = None, None

# Comparison
if r50_latency and r18_latency:
    speedup = r50_latency / r18_latency
    print(f"\n{'='*60}")
    print("COMPARISON")
    print(f"{'='*60}")
    print(f"ResNet18 is {speedup:.2f}x faster than ResNet50")
    print(f"ResNet18 has {r50_params / r18_params:.2f}x fewer parameters")
    print(f"\nProduction Constraint: <30ms")
    print(f"  ResNet50: {r50_latency:.2f}ms {'✓ PASS' if r50_latency < 30 else '✗ FAIL'}")
    print(f"  ResNet18: {r18_latency:.2f}ms {'✓ PASS' if r18_latency < 30 else '✗ FAIL'}")

## Final Summary: Experiment Results

Compare all experiments against the baseline

In [ ]:
import pandas as pd

print("\n" + "="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)

# Create results table
results_data = {
    "Experiment": [
        "Baseline (Production)",
        "Exp 1: Differential LR (ResNet50)",
        "Exp 2: Optimized Threshold",
        "Exp 3: ResNet18 Backbone"
    ],
    "Accuracy": [
        "92.99%",
        f"{best_val_acc:.2%}" if 'best_val_acc' in locals() else "N/A",
        "See threshold analysis",
        "TBD"
    ],
    "Latency (CPU)": [
        "~16ms",
        f"{r50_latency:.1f}ms" if r50_latency else "N/A",
        "Same as Exp 1",
        f"{r18_latency:.1f}ms" if r18_latency else "N/A"
    ],
    "Parameters": [
        "25.6M",
        "25.6M",
        "25.6M",
        "11.7M"
    ],
    "Status": [
        "Current Production",
        "✓ Complete" if 'best_val_acc' in locals() else "Pending",
        "✓ Complete",
        "✓ Complete" if r18_latency else "Pending"
    ]
}

df = pd.DataFrame(results_data)
print("\n")
print(df.to_string(index=False))

print("\n" + "="*60)
print("RECOMMENDATIONS FOR DEVELOPER MEETING")
print("="*60)

print("""
1. ACCURACY IMPROVEMENT:
   - Differential LR training showed improvement over baseline
   - Consider deploying if >94% accuracy achieved

2. THRESHOLD OPTIMIZATION:
   - Review threshold_analysis plots to find optimal balance
   - Recommend threshold that achieves ≥95% pit recall
   - Document acceptable false positive rate with stakeholders

3. MODEL SIZE/SPEED:
   - If ResNet18 achieves similar accuracy, recommend for deployment
   - Faster inference = higher throughput or lower hardware requirements
   - Smaller model = easier deployment and updates

4. FURTHER IMPROVEMENTS TO DISCUSS:
   - Collect more recent training data (if available)
   - Test with 224×224 input resolution (better feature extraction)
   - Explore ensemble methods (combine multiple models)
   - Add cherry size as an additional feature
""")

print("\nAll experiment outputs saved to Google Drive:")
print(f"  - Differential LR: {drive_output}")
print(f"  - Threshold Analysis: {threshold_output}")
print(f"  - ResNet18: {drive_output_r18}")